In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import argparse
import os
import pandas as pd
from datetime import datetime
from functools import partial
import numpy as np
import joblib
import tensorflow as tf
from tensorflow import keras
import sys

assert tf.__version__ >= "2.0"

# To make the output stable across runs
tf.random.set_seed(42)

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE_PER_REPLICA = 64
!nvidia-smi


Sun Dec 11 05:22:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
def create_model(kernel_size):
    DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=kernel_size,
                            activation='relu',
                            padding="VALID")

    return keras.models.Sequential([
        DefaultConv2D(filters=64, input_shape=[34, 121, 1]),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.5),

        # DefaultConv2D(filters=64),
        DefaultConv2D(filters = 256),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.5),

        # DefaultConv2D(filters=64),
        DefaultConv2D(filters = 256),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.5),

        DefaultConv2D(filters=32),
        # DefaultConv2D(filters = 64),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.5),

        keras.layers.Flatten(),

        keras.layers.Dense(units=256, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),

        keras.layers.Dense(units=2, activation='softmax'),
    ])



In [4]:
dataset_path = '/content/drive/MyDrive/Mahjong_data/reach_data.csv'
logs_path = '/content/drive/MyDrive/Training_log '
cnn_path = '/content/drive/MyDrive/CNN_model'


In [5]:
def data_format(data34):
    res = np.array([0]*34)
    for i in range(34):
        res[i] =  1 if (1<<(34-i -1)) & data34!= 0 else 0
    return res

In [6]:
data_all = pd.read_csv(dataset_path,header = None)
data_all.shape

(57217, 122)

In [7]:
def modify_dataframe(df):
    np_df = df.to_numpy()
    new_data = []
    for i in range(np_df.shape[0]):
        tmp = np.array([[0]*34]*np_df.shape[1])
        for j in range(np_df.shape[1]):
            tmp[j] = data_format(np_df[i][j])
        new_data.append(tmp.reshape(34,(np_df.shape[1]),1))
    return np.array(new_data)
        

In [8]:
def to_1x2_train(df):
    np_df = df.to_numpy()
    res = []
    for i in range(np_df.shape[0]):
        tmp = np.array([0]*2)
        if np_df[i][0] != 0 :
            tmp [0] = 1
        else :
            tmp [1] = 1
        res.append(tmp)
    return np.array(res)

In [9]:
X_train = modify_dataframe(data_all.iloc[:45000,:-1])
X_dev = modify_dataframe(data_all.iloc[45000:, :-1])

In [10]:
y_train = to_1x2_train(data_all.iloc[:45000, -1:])
y_dev = to_1x2_train(data_all.iloc[45000:, -1:])

In [11]:
y_train[0]

array([1, 0])

In [12]:
def train(custom_name,kernel_size = (3,1), custom_epochs = 25):
    tf.keras.backend.clear_session()
    strategy = tf.distribute.MirroredStrategy()
    num_of_gpus = strategy.num_replicas_in_sync
    print('Number of devices:', num_of_gpus)
    print()

    with strategy.scope():
        model = create_model(kernel_size)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        print(model.summary())
        print()

    # Train the neural network
    model_name = 'riichi_cnn_' + str(kernel_size[0]) + str(kernel_size[1]) + custom_name
    log_dir = os.path.join(logs_path + '/' + model_name + '/tensorboard_logs',
                           datetime.now().strftime("%Y%m%d-%H%M%S"))
    checkpoint_prefix = os.path.join(logs_path + '/' + model_name 
                                     + '/training_checkpoints',
                                     "checkpoint_{epoch}")

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=log_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                           save_best_only=True,
                                           save_weights_only=True,
                                           period=10)
    ]

    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * num_of_gpus
    history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=custom_epochs,
                        validation_data=(X_dev, y_dev),
                        callbacks=callbacks)

    # Save the neural network
    model.save(os.path.join(cnn_path, model_name + '.h5'))
    print('Save success!!!')
    eval_train = model.evaluate(X_train, y_train)
    print('final training loss:', eval_train[0])
    print('final training accuracy:', eval_train[1])
    eval_dev = model.evaluate(X_dev, y_dev)
    print('final dev loss:', eval_dev[0])
    print('final dev accuracy:', eval_dev[1])

In [13]:
train('_final_v2_10w_data_4conv',(3,1),5)

Number of devices: 1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 121, 64)       256       
                                                                 
 batch_normalization (BatchN  (None, 32, 121, 64)      256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 121, 256)      49408     
                                                                 
 batch_normalization_1 (Batc  (None, 30, 121, 256)     1024      
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 121, 256)      196864    
                                                                 
 batch_normalization_2 (Batc  (Non

None

Epoch 1/5
704/704 [==============================] - 145s 188ms/step - loss: 0.0021 - accuracy: 0.9992 - val_loss: 1.9903e-05 - val_accuracy: 1.0000
Epoch 2/5
704/704 [==============================] - 133s 188ms/step - loss: 6.4819e-06 - accuracy: 1.0000 - val_loss: 4.0268e-07 - val_accuracy: 1.0000
Epoch 3/5
704/704 [==============================] - 133s 188ms/step - loss: 2.4655e-06 - accuracy: 1.0000 - val_loss: 1.7347e-07 - val_accuracy: 1.0000
Epoch 4/5
704/704 [==============================] - 133s 189ms/step - loss: 3.5972e-06 - accuracy: 1.0000 - val_loss: 158.2010 - val_accuracy: 0.5618
Epoch 5/5
704/704 [==============================] - 133s 188ms/step - loss: 9.8513e-05 - accuracy: 1.0000 - val_loss: 2.4882e-08 - val_accuracy: 1.0000
Save success!!!
1407/1407 [==============================] - 39s 28ms/step - loss: 4.6971e-04 - accuracy: 1.0000
final training loss: 0.0004697144322562963
final training accuracy: 0.9999777674674988
382/382 [==========================